In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

for dirname, _, filenames in os.walk("./"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. Frame the problem
Using the customer description, Define the problem your trying to solve in your own words (remember this is not technial but must be specific so the customer understands the project

We are tasked with creating an automated tetris player that can successfully play for an extended period of time. By using a Reinforcement Learning model we will achieve this goal with the several characteristics we can define and configure within the model to make it properly play the game.

# 2. Get the Data 
Define how you recieved the data (provided, gathered..)

The data is collected by real-time training through trial and error, the model observes the current state takes and action, then recieves either positive or negative feedback. Data is collected by completing several games overtime. 

# 3. Explore the Data
Gain insights into the data you have from step 2, making sure to identify any bias

The board consists of a 2D grid that is 10 x 20 that can take in binary values (0 = empty, 1 = filled). There are 7 types of blocks that will be taken into account along with the current roation state of those shapes. Some of the features that will be derived is the board height before each piece is placed, number of holes, bumpiness, number of complete lines, and the individual column heights. The model will be able to rotate the piece 0, 80, 180, or 270 degrees and move its horizontal position as long as it's within the confinments of the board. 

# 4.Prepare the Data


Apply any data transformations and explain what and why


To properly prepare the data we will need to flatten the 10x20 board to a 200-dimensional vector and keep the normslization of 0 for empty 1 for filled. Then feature engineering will include the hand-extracted features. We then need to scale the rewards to a resonable range where positive awards outweigh the negative awards to fuel efficiency in a Reinforcement Learning model. 

# 5. Model the data
Using selected ML models, experment with your choices and describe your findings. Finish by selecting a Model to continue with


I first attempted creating a DQN model however it came to my attention that not only was each generation very slow but it kept overfitting and learning mostly from failures and little to no successes so the results were very sparse and it never properly learned. I ended up using a genetic algorithm with heuristic weights which worked much better due to the immediate feedback and the heuristic weights were much more comparable to the factors that play into Tetris those being height, holes, bumpiness, etc... after each generation the model improves even slightly and trains much faster and efficiently.  

# 6. Fine Tune the Model

With the select model descibe the steps taken to acheve the best rusults possiable 


In [ ]:
"""
Clean Evolutionary Tetris AI - Starting Fresh
Simple, working evolutionary algorithm
"""
import random
import pickle
import os

# ============================================================================
# CONFIGURATION
# ============================================================================
TRAINING_MODE = True
CHECKPOINT_DIR = "checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Evolution settings
POPULATION_SIZE = 20
ELITE_SIZE = 4
MUTATION_RATE = 0.3
MUTATION_STRENGTH = 0.2
GAMES_PER_INDIVIDUAL = 3

# ============================================================================
# BOARD EVALUATION
# ============================================================================
def evaluate_board(board_array):
    """Calculate board features"""
    height = len(board_array)
    width = len(board_array[0])
    
    # Column heights
    heights = []
    for col in range(width):
        col_height = 0
        for row in range(height):
            if board_array[row][col]:
                col_height = row + 1
        heights.append(col_height)
    
    # Features
    aggregate_height = sum(heights)
    max_height = max(heights) if heights else 0
    
    # Holes
    holes = 0
    for col in range(width):
        found_block = False
        for row in range(height - 1, -1, -1):
            if board_array[row][col]:
                found_block = True
            elif found_block:
                holes += 1
    
    # Bumpiness
    bumpiness = sum(abs(heights[i] - heights[i + 1]) for i in range(len(heights) - 1))
    
    # Complete lines
    complete_lines = sum(1 for row in board_array if all(row))
    
    # Wells
    wells = 0
    for i in range(width):
        if i == 0:
            if heights[i] < heights[i + 1]:
                wells += heights[i + 1] - heights[i]
        elif i == width - 1:
            if heights[i] < heights[i - 1]:
                wells += heights[i - 1] - heights[i]
        else:
            if heights[i] < heights[i - 1] and heights[i] < heights[i + 1]:
                wells += min(heights[i - 1], heights[i + 1]) - heights[i]
    
    return {
        'aggregate_height': aggregate_height,
        'max_height': max_height,
        'holes': holes,
        'bumpiness': bumpiness,
        'complete_lines': complete_lines,
        'wells': wells
    }

def simulate_placement(board, piece, x):
    """Simulate placing a piece"""
    board_copy = [row[:] for row in board.board[:board.height]]
    
    try:
        y = board.drop_height(piece, x)
    except:
        return None
    
    for pos in piece.body:
        row_idx = y + pos[1]
        col_idx = x + pos[0]
        if 0 <= row_idx < board.height and 0 <= col_idx < board.width:
            board_copy[row_idx][col_idx] = True
    
    return board_copy

# ============================================================================
# EVOLUTIONARY AGENT
# ============================================================================
class Agent:
    def __init__(self):
        # Start with reasonable initial weights
        self.best_weights = {
            'aggregate_height': -0.5,
            'max_height': -0.5,
            'holes': -1.0,
            'bumpiness': -0.3,
            'complete_lines': 0.8,
            'wells': -0.5
        }
        
        self.generation = 0
        self.best_score = 0
        self.population = []
        
        # Try to load checkpoint
        checkpoint_path = os.path.join(CHECKPOINT_DIR, "evolution.pkl")
        if os.path.exists(checkpoint_path):
            try:
                with open(checkpoint_path, 'rb') as f:
                    data = pickle.load(f)
                self.best_weights = data['best_weights']
                self.generation = data['generation']
                self.best_score = data['best_score']
                print(f"Loaded checkpoint: Gen {self.generation}, Score {self.best_score}")
            except:
                print("Checkpoint corrupted, starting fresh")
        
        # Create population
        self.create_population()
    
    def create_population(self):
        """Create population of weight variations"""
        self.population = []
        
        # First individual: best weights
        self.population.append({
            'weights': self.best_weights.copy(),
            'scores': []
        })
        
        # Rest: variations
        for _ in range(POPULATION_SIZE - 1):
            weights = {}
            for key, value in self.best_weights.items():
                weights[key] = value + random.gauss(0, 0.5)
            self.population.append({
                'weights': weights,
                'scores': []
            })
    
    def evaluate_move(self, board, piece, x, weights):
        """Evaluate a move"""
        simulated_board = simulate_placement(board, piece, x)
        if simulated_board is None:
            return -999999
        
        features = evaluate_board(simulated_board)
        score = sum(weights[k] * features[k] for k in weights)
        return score
    
    def get_best_move(self, board, piece, weights):
        """Find best move for given weights"""
        best_x = 0
        best_piece = piece
        best_score = -999999
        
        current_piece = piece
        for _ in range(4):
            for x in range(board.width):
                if x + len(current_piece.skirt) > board.width:
                    continue
                
                try:
                    score = self.evaluate_move(board, current_piece, x, weights)
                    if score > best_score:
                        best_score = score
                        best_x = x
                        best_piece = current_piece
                except:
                    pass
            
            current_piece = current_piece.get_next_rotation()
        
        return best_x, best_piece
    
    def evolve(self):
        """Evolve to next generation"""
        # Calculate average score for each individual
        for ind in self.population:
            if ind['scores']:
                ind['avg_score'] = sum(ind['scores']) / len(ind['scores'])
            else:
                ind['avg_score'] = 0
        
        # Sort by score
        self.population.sort(key=lambda x: x['avg_score'], reverse=True)
        
        # Update best
        if self.population[0]['avg_score'] > self.best_score:
            self.best_score = self.population[0]['avg_score']
            self.best_weights = self.population[0]['weights'].copy()
        
        # Keep elite
        new_population = []
        for i in range(ELITE_SIZE):
            new_population.append({
                'weights': self.population[i]['weights'].copy(),
                'scores': []
            })
        
        # Create offspring
        while len(new_population) < POPULATION_SIZE:
            # Select parents from elite
            p1 = random.choice(self.population[:ELITE_SIZE])
            p2 = random.choice(self.population[:ELITE_SIZE])
            
            # Crossover
            child_weights = {}
            for key in self.best_weights.keys():
                child_weights[key] = p1['weights'][key] if random.random() < 0.5 else p2['weights'][key]
                
                # Mutation
                if random.random() < MUTATION_RATE:
                    child_weights[key] += random.gauss(0, MUTATION_STRENGTH)
            
            new_population.append({
                'weights': child_weights,
                'scores': []
            })
        
        self.population = new_population
        self.generation += 1
        
        # Save checkpoint
        checkpoint_path = os.path.join(CHECKPOINT_DIR, "evolution.pkl")
        with open(checkpoint_path, 'wb') as f:
            pickle.dump({
                'best_weights': self.best_weights,
                'generation': self.generation,
                'best_score': self.best_score
            }, f)

# ============================================================================
# MAIN CLASS
# ============================================================================
_agent = None
_current_ind = 0
_games_played = 0

class CUSTOM_AI_MODEL:
    def __init__(self):
        global _agent, _current_ind
        
        if _agent is None:
            _agent = Agent()
            print(f"\nEvolutionary Tetris AI")
            print(f"Generation: {_agent.generation}")
            print(f"Population: {POPULATION_SIZE}")
            print(f"Best score: {_agent.best_score}\n")
        
        self.agent = _agent
        
        # Get weights for current individual
        if _current_ind < len(self.agent.population):
            self.weights = self.agent.population[_current_ind]['weights']
        else:
            self.weights = self.agent.best_weights
    
    def get_best_move(self, board, piece, depth=1):
        """Called by game.py"""
        x, selected_piece = self.agent.get_best_move(board, piece, self.weights)
        return x, selected_piece
    
    def __del__(self):
        """Called when game ends"""
        global _current_ind, _games_played
        
        if not TRAINING_MODE:
            return
        
        # Continue training
        from game import Game
        
        try:
            while True:
                # Run game
                game = Game("student")
                pieces, rows = game.run_no_visual()
                
                # Calculate score
                score = pieces + rows * 100
                
                # Record score
                if _current_ind < len(self.agent.population):
                    self.agent.population[_current_ind]['scores'].append(score)
                
                _games_played += 1
                
                # Print simple output
                print(f"{pieces} {rows}")
                
                # Check if individual is done
                if _current_ind < len(self.agent.population):
                    if len(self.agent.population[_current_ind]['scores']) >= GAMES_PER_INDIVIDUAL:
                        _current_ind += 1
                
                # Check if generation is done
                if _current_ind >= POPULATION_SIZE:
                    # Evolve
                    self.agent.evolve()
                    _current_ind = 0
                    
                    # Print generation summary
                    print(f"\n=== Generation {self.agent.generation} ===")
                    print(f"Best score: {self.agent.best_score:.0f}")
                    print(f"Best weights: {self.agent.best_weights}")
                    print()
                
        except KeyboardInterrupt:
            print(f"\nStopped at generation {self.agent.generation}")
            import sys
            sys.exit(0)


# 7. Present
In a customer faceing Document provide summery of finding and detail approach taken


The model employs a genetic algorithm to evolve certain heuristic weights for a board evaluation function. For each piece placement decision, the algorithm simulates all possible positions and rotations in a game of Tetris, it then evaluates each resulting board state using six weighted features (aggregate height, maximum height, holes, bumpiness, complete lines, and wells), and selects the placement with the highest weighted score. The weights are evolved over generations using a population of 20 individuals, where each individual represents a unique weight tested across 5 games to compute average fitness. After evaluation, the top 4 generations are preserved, while the remaining population is generated through crossover and mutation of the elite parents. This process iterates across generations, with the target network updating after each generation to incorporate the best-performing weights.

# 8. Launch the Model System
Define your production run code, This should be self susficent and require only your model pramaters 


In [ ]:
import sys
import os

sys.path.insert(0, 'src')

from game import Game

try:
    g = Game("student")
    g.run_no_visual()
except KeyboardInterrupt:
    print("\nTraining stopped")
    print("Checkpoint saved to: checkpoints/evolution.pkl")
